# What
分类任务，支持两种模式
1. Folder模式，需要输入`train`, `valid`两个测试集对应的目录。`labels.txt`，需要训练的label，里面每个类别一行。
2. List模式，需要输入`train`, `valid`两个测试集对应的训练文件，每行一个样本。`labels.txt`是可选参数，里面每个类别一行。`data_pattern`一个通用的目录，与train、val中的第一列进行拼接。

In [ ]:
import os
from onekey_algo.classification.run_classification import main as clf_main
from collections import namedtuple
from onekey_algo import get_param_in_cwd

# 设置参数
crop_mapping = {'MR-CE': 'MR-CE/vnet_crop', 'MR-T2': 'MR-T2/vnet_crop', 'endoscope': 'Segmentation\endoscope_coco/fcn_crop',
                'MR-CE_anno': 'MR-CE/crop', 'MR-T2_anno': 'MR-T2/crop', 'endoscope_anno': 'Segmentation\endoscope_coco/crop'}
crop_mapping = {'endoscope_anno': 'Segmentation\endoscope_coco/crop'}
root = get_param_in_cwd('radio_dir')
for modal in crop_mapping.keys():
    for cv in range(6, 7):
        for model_name in ['resnet18', 'resnet101', 'densenet121', 'TwinsSVT'][2:3]:
            train_f = os.path.join('split_info', f'train-RND-{cv}.txt')
            val_f = os.path.join('split_info', f'val-RND-{cv}.txt')
            labels_f = os.path.join('split_info', 'labels.txt')
            data_pattern = os.path.join(root, crop_mapping[modal])
            params = dict(train=train_f,
                          valid=val_f,
                          labels_file=labels_f,
                          in_channels=3,
                          data_pattern=data_pattern,
                          j=4,
                          max2use=None,
                          val_max2use=None,
                          batch_balance=False,
                          normalize_method='imagenet',
                          model_name=model_name,
                          gpus=[0],
                          batch_size=16,
                          epochs=48 if model_name not in ['TwinsSVT', 'CrossFormer', 'SimpleViT'] else 300,
                          init_lr=0.01 if model_name not in ['TwinsSVT', 'CrossFormer', 'SimpleViT'] else 0.005,
                          optimizer='sgd',
                          retrain=r'',
                          model_root= os.path.join(root, 'Classification', modal, f"CV-{cv}"),
                          vit_settings = {'patch_size': 64, 'dim': 1024, 'depth': 6, 'heads': 16, 'mlp_dim': 2048},
                          iters_start=0,
                          iters_verbose=4,
                          save_per_epoch=True,
                          add_date=False,
                          pretrained=False)
            # 训练模型
            Args = namedtuple("Args", params)
            clf_main(Args(**params))